In [1]:
import torch
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch import nn
import numpy as np
import math

In [2]:
class cnnEvalModel():
    def __init__(self, modelFilePath):

        self.images = None
        self.labels = None
        self.logps = None
        self.output = None
        self.pred = None
        self.img = None
        self.imgOut = None

        # torch.manual_seed(0)  # set for repeatable result
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        np.random.seed(0)

        self.device = torch.device("cpu")
        print(self.device)

        mean, std = (0.5,), (0.5,)

        # Create a transform and normalise data
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(mean, std)
                                      ])
        # Download FMNIST test dataset and load test data
        self.testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
        self.testloader = torch.utils.data.DataLoader(self.testset, batch_size=1, shuffle=True)

        self.modelPath = modelFilePath

        self.model = torch.jit.load(self.modelPath, self.device)
        self.model.eval()

    def denormalize(tensor):
        """denormalizes a supplied tensor and returns it"""
        tensor = tensor*0.5 + 0.5
        return tensor

    def runModel(self, count):
        """Runs the cnn model based on the iteration count for test set and converts model tensor outputs
        to regular python data types
            Accepts count as an integer"""

        tempCount = 0
        # iterating to the number of counts in arg count
        for batch, (tempImg, tempLabel) in enumerate(self.testloader, 1):
            
            tempCount += 1
            
            if tempCount == count:
                self.images = tempImg
                self.labels = tempLabel

                break

        # send image and label tensors to GPU memory
        self.images = self.images.to(self.device)
        self.labels = self.labels.to(self.device)

        # run model on loaded image and label data
        self.logps = self.model(self.images)

        # parse model output
        self.output = torch.exp(self.logps)
        self.pred = torch.argmax(self.output, 1)

        # copy 1-D tensor to cpu memory and convert to python integer
        self.pred = self.pred.item()
        self.labels = self.labels.item()

        # copy image data to cpu memory
        self.img = self.images

    def saveImagePlotData(self):
        """processes and saves image tensor data in a .png file"""
        self.imgOut = self.img.view(28, -1)
        self.imgOut = cnnEvalModel.denormalize(self.imgOut)

        save_image(self.imgOut, 'tensorImage.png')

In [3]:
filePath = 'best_model_acc_2convL_2FCL_9178.pt'
cnnModel = cnnEvalModel(filePath)

cpu


In [4]:
import ipywidgets as widgets
from ipycanvas import Canvas
from IPython.display import display

In [18]:
def createButton(buttonText, buttonStyle):
    """ Creates a ipywidgets button object with a button style from the default button style strings 
    Accepts buttonText as a string
            buttonStyle as a string
    Returns theButton as a ipywidgets button object """
    
    theButton = widgets.Button(description=buttonText,
                               button_style=buttonStyle,
                               layout=widgets.Layout(width='auto',
                                                     height='auto'))
    return theButton

def createLabel(labelText, leftPadding, horzPadAmnt, topPadding, vertPadAmnt):
    """ Creates a ipywidgets label object with padding applied depending on supplied amounts.
    Accepts labelText as a string
            leftPadding as a boolean if left padding is used, right padding otherwise
            horzPadAmnt as a string of padding amount in format '10%'
            topPadding as a boolean if top padding is used, bottom padding otherwise
            verPadAmnt as a string of padding amount in format '10%'
    Returns theLabel as a ipywidgets label object """
    
    if leftPadding:
        if topPadding:
            theLabel = widgets.Label(value=labelText,
                                     layout=widgets.Layout(left=horzPadAmnt, 
                                                           top=vertPadAmnt))
        else:
            theLabel = widgets.Label(value=labelText,
                         layout=widgets.Layout(left=horzPadAmnt,
                                               bottom=vertPadAmnt))
    else:
        if topPadding:
            theLabel = widgets.Label(value=labelText,
                                     layout=widgets.Layout(right=horzPadAmnt,
                                                           top=vertPadAmnt))
        else:
            theLabel = widgets.Label(value=labelText,
                         layout=widgets.Layout(right=horzPadAmnt,
                                               bottom=vertPadAmnt))

    return theLabel

class UI():
    def __init__(self, imageFileName):
        # initialize the model
        cnnModel.runModel(1)
        cnnModel.saveImagePlotData()
        
        # initialize needed variables for tracking and UI creation
        self.count = 2        # sets the image # for testing dataset
        self.clothingTypes = ['t-shirt/top', 'pants/trousers', 'pullover/sweater', 'dress', 'coat/jacket',
                      'sandal/open-toe', 'shirt/button-up shirt', 'sport/casual shoe', 'bag/purse', 'ankle boot']
        self.correctChoice = None
        self.correctAnsColor = [(0, 0, 0), (105, 229, 20), (0, 0, 0)]
        self.wrongAnsColor = [(0, 0, 0), (247, 27, 27), (0, 0, 0)]
        self.blackColor = [(0, 0, 0), (0, 0, 0), (0, 0, 0)]
        self.radiansBarCount = 0.0
        self.imageFileNames = imageFileName
        
        # score tracking variables
        self.iterationNum = 0.0
        self.userCorrect = 0.0
        self.cpuCorrect = 0.0
        self.userCorrectPercent = 0.0
        self.cpuCorrectPercent = 0.0
        self.oldUserPercent = 0.0

        
        # grab the tensor image
        file = open(self.imageFileNames, 'rb')
        image = file.read()

        # create image widget
        self.imageWidget = widgets.Image(
            value = image,
            format = 'png',
            width = 450,
            height = 450,
        )
        
        # make buttons
        self.buttons = [''] * 10
        
        for i in range(10):
            self.buttons[i] = createButton(self.clothingTypes[i], 'info')
            self.buttons[i].on_click(self.on_button_clicked)
            
        # make labels
        self.lastCorrectLabelStatic = createLabel('Correct clothing from last image:', True, '0%', True, '0%')
        self.lastCorrectLabelVar = createLabel('0', False, '60%', True, '0%')

        self.userScoreLabel = createLabel('Your Score:', True, '30%', True, '23%')
        self.userScorePercentLabel = createLabel('00.00%', True, '35%', True, '20%')
        self.cpuScoreLabel = createLabel('DL Model Score:', True, '20%', True, '23%')
        self.cpuScorePercentLabel = createLabel('00.00%', True, '35%', True, '20%')

        self.userInfoStaticLabel = createLabel('Correct images:', True, '22%', True, '0%')
        self.userInfoDynamicLabel = createLabel('0', False, '65%', True, '0%')
        self.cpuInfoStaticLabel = createLabel('DL model correct images:', True, '20%', True, '0%')
        self.cpuInfoDynamicLabel = createLabel('0', True, '20%', True, '0%')
        self.numImagesStaticLabel = createLabel('Total images:', True, '45%', True, '0%')
        self.numImagesDynamicLabel = createLabel('0', True, '0%', True, '0%')

        # make progress bars
        self.userCanvas = Canvas(width=420,height=120)
        self.userPolygons = [[(0, 10), (2, 10), (2, 50), (0, 50)], 
                             [(100, 10), (200, 10), (200, 50), (100, 50)],
                             [(0, 70), (2, 70), (2, 110), (0, 110)]]
        self.userPolygonsColors = [(0, 0, 0), (255, 255, 255), (0, 0, 0)]
        self.userCanvas.fill_styled_polygons(self.userPolygons, color=self.userPolygonsColors)
        
        # create grid object
        self.grid = widgets.GridspecLayout(30, 35, width='700px', height='900px')
        
        #apply widgets to grid
        self.grid[1:17, 6:31] = self.imageWidget

        self.grid[17, :14] = self.lastCorrectLabelStatic
        self.grid[17, 14:21] = self.lastCorrectLabelVar
        
        # top row of buttons
        self.grid[18:20, :7] = self.buttons[0]
        self.grid[18:20, 7:14] = self.buttons[1]
        self.grid[18:20, 14:21] = self.buttons[2]
        self.grid[18:20, 21:28] = self.buttons[3]
        self.grid[18:20, 28:35] = self.buttons[4]

        # bottom row of buttons
        self.grid[20:22, :7] = self.buttons[5]
        self.grid[20:22, 7:14] = self.buttons[6]
        self.grid[20:22, 14:21] = self.buttons[7]
        self.grid[20:22, 21:28] = self.buttons[8]
        self.grid[20:22, 28:35] = self.buttons[9]

        self.grid[23:25, :7] = self.userScoreLabel
        self.grid[23:27, 7:28] = self.userCanvas
        self.grid[23:25, 28:] = self.userScorePercentLabel

        self.grid[25:27, :7] = self.cpuScoreLabel
        # self.grid[25:27, 7:28] = self.cpuCanvas
        self.grid[25:27, 28:] = self.cpuScorePercentLabel

        self.grid[28, :9] = self.userInfoStaticLabel
        self.grid[28, 9:12] = self.userInfoDynamicLabel
        self.grid[28, 12:21] = self.cpuInfoStaticLabel
        self.grid[28, 21:24] = self.cpuInfoDynamicLabel
        self.grid[28, 24:32] = self.numImagesStaticLabel
        self.grid[28, 32:] = self.numImagesDynamicLabel

        display(self.grid)
        
    def on_button_clicked(self, b):
        """ Called upon during a button click event to run the update on UI window.
        Accepts b as the button object passed from the button's .on_click method """
        
        self.updateUI(b)
        
    def getButtonID(self, buttonObj):
        """ Uses the description of the passed button object to determine and return the button's number.
        Accepts buttonObj as a button object
        Returns buttonNumber as an int """
        
        buttonDescrip = buttonObj.description
        
        for i in range(10):
            if buttonDescrip == self.clothingTypes[i]:
                buttonNumber = i
        
        return buttonNumber
        
    def updateScoreStrVar(self, scorePercent):
        """ Converts a float into a percent string rounded to 2 decimal places.
        Accept scorePercent as a float
        Returns tempScoreString as a string """
        
        if scorePercent > 100:
            tempScoreString = '100.00%'
        elif scorePercent <= 0:
            tempScoreString = '0.00%'
        else:
            tempScoreString = str(round(scorePercent, 2)) + '%'
            
        return tempScoreString
    
    def updateScoreBarPixel(self, scorePercent):
        """ Uses the score percent and returns the width of the score bar in pixels
        for drawing ipycanvas canvas object shapes. (used for cpu score bar)
        Accepts scorePercent as a float
        Returns tempScoreWidth as an int """

        if scorePercent > 100:      # deal with edge cases
            tempScoreWidth = 420
        elif scorePercent < (100/420):    # deal with edge cases
            tempScoreWidth = 2
        else:
            tempScoreWidth = round(scorePercent / 100 * 420)
        print(tempScoreWidth)
        return tempScoreWidth
    
    def updateScoreBarPosition(self, scorePercent, oldScorePercent):
        """ Uses the score percent and returns the width of two score bars. scoreBlackBarX2Pos is the base score bar width
        and scoreColourBarX2Pos is the width of the colored score bar showing the change in the score bar from the previous image.
        Accepts scorePercent as a float
                oldScorePeercent as a float
        Returns scoreBlackBarX2Pos as an int
                scoreColourBarX2Pos as an int
                significantChange as a boolean """
        
        if scorePercent > 100:      # deal with edge cases
            scoreWidthDefault = 420
        elif scorePercent < (100 / 420):    # deal with edge cases
            scoreWidthDefault = 1
        else:
            scoreWidthDefault = round(scorePercent / 100 * 420)

        if abs(scorePercent - oldScorePercent) >= (100 / 420):      # check for change in percent between old and new percent
            significantChange = True 

            if scorePercent > oldScorePercent:       # check if score has increased/decreased
                scoreBlackBarX2Pos = round(oldScorePercent / 100 * 420)
                scoreColourBarX2Pos = round(scorePercent / 100 * 420)
            else:
                scoreBlackBarX2Pos = round(scorePercent / 100 * 420)
                scoreColourBarX2Pos = round(oldScorePercent / 100 * 420)

        else:       # assigns x-position values if score percentage stays at 100% or 0%
            significantChange = False
            scoreBlackBarX2Pos = scoreWidthDefault
            scoreColourBarX2Pos = scoreWidthDefault + 1

        return scoreBlackBarX2Pos, scoreColourBarX2Pos, significantChange

    def cosineColorVal(self, startColor, endColor, radians):
        """ Takes a starting and ending color value and returns a new color value that is between the start and end
        defined by a cosine function.
        Accepts startColor as a 3x1 integer tuple
                endColor as a 3x1 integer tuple
                radians as a float
        Returns tempColor as an integer tuple of form (x, x, x) """
        
        tempColor = [0, 0, 0]
        
        for i in range(3):
            tempColor[i] = round(startColor[i] + (endColor[i] - startColor[i]) * (math.cos(radians) + 1) / 2)
            
        tempColor = tuple(tempColor)
        return tempColor
    
    def scoreBarColorFade(self, startColor, endColor, radians, shapes):
        """ Takes a given ipycanvas shape and creates a gradient color change by recursively redrawing the shape
        using redrawUserPolygons() with new color values using cosineColorVal().
        Accepts startColor as a 3x1 integer tuple
                endColor as a 3x1 integer tuple
                radians as a float
                shapes as a list of four 2x1 integer tuples """
        
        if radians < math.pi*2:
            newColor = self.cosineColorVal(startColor, endColor, radians)
            self.radiansBarCount += math.pi / 100
            self.redrawUserPolygons(shapes, [(0, 0, 0), newColor, (0, 0, 0)])
            self.scoreBarColorFade(newColor, endColor, self.radiansBarCount, shapes)
        else:
            return
        
    def redrawUserPolygons(self, shapes, colors):
        """ Redraws the shapes on an ipycanvas canvas using supplied list of coords and rgb color value.
        Accepts shapes as a list of four 2x1 integer tuples
                colors as a 3x1 integer tuple """
        
        # self.userCanvas.clear_rect(0, 0, 420, 60)
        self.userCanvas.fill_styled_polygons(shapes, colors)
    
    def updateUI(self, buttonObj):
        """ Updates UI window on button press - calculates correct percentages, redraws score bars, 
        refreshes tracking labels, and updates the model and image for the next iteration.
        Accepts buttonObj as a button object """
        
        self.count += 1
        self.iterationNum += 1
        
        # get correct answer and cpu+user prediction
        correctChoice = cnnModel.labels
        cpuChoice = cnnModel.pred
        userChoice = self.getButtonID(buttonObj)
        
        if userChoice == correctChoice:
            self.userCorrect += 1 
            self.correctChoice = True
        else:
            self.correctChoice = False
        
        if cpuChoice == correctChoice:
            self.cpuCorrect +=  1
        
        # re-calculate user+cpu correct percentages
        self.userCorrectPercent = self.userCorrect / self.iterationNum * 100
        self.cpuCorrectPercent = self.cpuCorrect / self.iterationNum * 100
        
        # update score bars
        blackBarX2Pos, colorBarX2Pos, significantChanges = self.updateScoreBarPosition(self.userCorrectPercent, self.oldUserPercent)
        cpuBarX2Pos = self.updateScoreBarPixel(self.cpuCorrectPercent)
        
        self.polygons = [[(0, 10), (blackBarX2Pos, 10), (blackBarX2Pos, 50), (0, 50)], 
                             [(blackBarX2Pos, 10), (colorBarX2Pos, 10), (colorBarX2Pos, 50), (blackBarX2Pos, 50)],
                             [(0, 70), (cpuBarX2Pos, 70), (cpuBarX2Pos, 110), (0, 110)]]
 
        self.oldUserPercent = self.userCorrectPercent
        
        # change color on right (green) or wrong (red)
        self.userCanvas.clear_rect(0, 0, 420, 120)
        if significantChanges:
            if self.correctChoice:
                self.radiansBarCount = math.pi
                self.redrawUserPolygons(self.polygons, self.correctAnsColor)
                self.scoreBarColorFade(self.correctAnsColor[1], self.correctAnsColor[0], self.radiansBarCount, self.polygons)
            else:
                self.radiansBarCount = math.pi
                self.redrawUserPolygons(self.polygons, self.wrongAnsColor)
                self.scoreBarColorFade(self.wrongAnsColor[1], (255, 255, 255), self.radiansBarCount, self.polygons)
        else:
            self.redrawUserPolygons(self.polygons, self.blackColor)
        
        # update model
        cnnModel.runModel(self.count)
        cnnModel.saveImagePlotData()
        file = open(self.imageFileNames, 'rb')
        image = file.read()
        self.imageWidget.value = image
        
        # update correct percentages
        self.grid[23:25, 28:] = createLabel(self.updateScoreStrVar(self.userCorrectPercent), True, '35%', True, '20%')
        self.grid[25:27, 28:] = createLabel(self.updateScoreStrVar(self.cpuCorrectPercent), True, '35%', True, '20%')
        
        # update image count labels
        self.grid[28, 9:12] = createLabel(str(round(self.userCorrect)), False, '65%', True, '0%')
        self.grid[28, 21:24] = createLabel(str(round(self.cpuCorrect)), True, '20%', True, '0%')
        self.grid[28, 32:] = createLabel(str(round(self.iterationNum)), True, '0%', True, '0%')
        
        # update last correct image clothing type
        self.grid[17, 14:21] = createLabel(self.clothingTypes[correctChoice], False, '62%', True, '0%') 



In [19]:
UIPane = UI('tensorImage.png')

GridspecLayout(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08…